# Imports and Setup

In [ ]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import splitfolders
from sklearn.metrics import classification_report

In [ ]:
components_list = ['COM_male', 'Display_Port', 'Ethernet', 'LED_HDD', 'LED_PWR', 'LED_SSD', 'Line_out', 'Loch', 'LOGO_AllenBradley_font', 'LOGO_AllenBradley_icon', 'LOGO_Kontron_font', 'LOGO_Kontron_icon', 'Power_5V', 'Powerstecker_off', 'Powerstecker_on', 'Reset_button', 'Schraube', 'Schraube_abstand', 'Schraube_gespiegelt', 'Schraube_halb', 'Schraube_rund', 'USB_2', 'USB_3', 'VGA']

For training and evaluation, Components_Binary.zip must be uploaded manually.
This was done, as loading the data directly from Google Drive took too long.

In [ ]:
!unzip 'Components_Binary.zip'

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: Components_Binary/COM_male/1/COM_male_784.jpg  
  inflating: __MACOSX/Components_Binary/COM_male/1/._COM_male_784.jpg  
  inflating: Components_Binary/COM_male/1/COM_male_4222.jpg  
  inflating: __MACOSX/Components_Binary/COM_male/1/._COM_male_4222.jpg  
  inflating: Components_Binary/COM_male/1/COM_male_4791.jpg  
  inflating: __MACOSX/Components_Binary/COM_male/1/._COM_male_4791.jpg  
  inflating: Components_Binary/COM_male/1/COM_male_3982.jpg  
  inflating: __MACOSX/Components_Binary/COM_male/1/._COM_male_3982.jpg  
  inflating: Components_Binary/COM_male/1/COM_male_1993.jpg  
  inflating: __MACOSX/Components_Binary/COM_male/1/._COM_male_1993.jpg  
  inflating: Components_Binary/COM_male/1/COM_male_3606.jpg  
  inflating: __MACOSX/Components_Binary/COM_male/1/._COM_male_3606.jpg  
  inflating: Components_Binary/COM_male/1/COM_male_1704.jpg  
  inflating: __MACOSX/Components_Binary/COM_male/1/._COM_male_1

# Data Loading and Preprocessing

In [ ]:
def load_data(component):
  # Split data into train, test and validation sets
  splitfolders.ratio(f'Components_Binary/{component}/', output=f'output_{component}', seed=1337, ratio=(0.7, 0.2, 0.1))

  # Create imagefolders and transform data
  train_folder = torchvision.datasets.ImageFolder(f'/content/output_{component}/train',
                                                  transform =  torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                                                              torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                                                              torchvision.transforms.RandomHorizontalFlip(),
                                                                                              torchvision.transforms.RandomVerticalFlip(),
                                                                                              torchvision.transforms.Resize((244,244))]))

  val_folder = torchvision.datasets.ImageFolder(f'/content/output_{component}/val',
                                                  transform =  torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                                                              torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                                                              torchvision.transforms.RandomHorizontalFlip(),
                                                                                              torchvision.transforms.RandomVerticalFlip(),
                                                                                              torchvision.transforms.Resize((244,244))]))

  test_folder = torchvision.datasets.ImageFolder(f'/content/output_{component}/test',
                                                  transform =  torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                                                              torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                                                              torchvision.transforms.RandomHorizontalFlip(),
                                                                                              torchvision.transforms.RandomVerticalFlip(),
                                                                                              torchvision.transforms.Resize((244,244))]))

  # Dataloaders
  train_dataloader = torch.utils.data.DataLoader(train_folder, batch_size=32, shuffle=True)
  val_dataloader = torch.utils.data.DataLoader(val_folder, batch_size=32)
  test_dataloader = torch.utils.data.DataLoader(test_folder, batch_size=32)

  return train_dataloader, val_dataloader, test_dataloader

# Training

In [ ]:
def train_model(net, train_dataloader, val_dataloader):

  # Training Loop
  net = net.to('cuda')

  optimizer = torch.optim.Adam(net.parameters(), lr=3e-4)
  loss_fct = nn.BCEWithLogitsLoss()

  train_losses = []
  val_losses = []
  for t in range(10):
    net.train()

    epoch_train_loss = 0.
    for X_batch, y_batch in train_dataloader:
      X_batch = X_batch.to('cuda')
      y_batch = y_batch.to('cuda')

      y_pred = net(X_batch)

      loss = loss_fct(y_pred.squeeze(), y_batch.float())
      epoch_train_loss += loss

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    train_losses.append(epoch_train_loss.cpu().data / len(train_dataloader))

    net.eval()
    epoch_val_loss = 0.
    for X_batch, y_batch in val_dataloader:
      X_batch = X_batch.to('cuda')
      y_batch = y_batch.to('cuda')
      with torch.no_grad():
        y_pred_val = net(X_batch)
        loss = loss_fct(y_pred_val.squeeze(), y_batch.float())
        epoch_val_loss += loss
    val_losses.append(epoch_val_loss.cpu().data / len(val_dataloader))

    # ---------> UNCOMMENT TO SAVE TRAINED MODEL WEIGHTS <---------
    #torch.save(net.state_dict(), f"/content/drive/MyDrive/DeepVis/PretrainedBinary/{component}.pth")

    return net

# Eval

In [ ]:
def evaluate_model(trained_model, test_dataloader, component):
  trained_model.eval()
  y_pred_list = []
  y_true_list = []
  for X_batch, y_batch in train_dataloader:
    X_batch = X_batch.to('cuda')
    y_pred = trained_model(X_batch)
    # The next line of code has to translate the single output of the neural network into a class prediction (0 or 1).
    # As the output are logits, we can simply say that an output of greater than 0 would translate into the model
    # predicting class 1, else it predicts class 0.
    y_pred_list.extend((1 * (y_pred.squeeze() > 0.0)).tolist())
    y_true_list.extend(y_batch.tolist())

  print(component)
  print(classification_report(y_true_list, y_pred_list))
  print('---------------------------------------------')

# Hand-crafted

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)

        self.fc1 = nn.Linear(in_features=107648, out_features=50)  
        # 1 output feature for binary classification
        self.fc2 = nn.Linear(in_features=50, out_features=1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        x = x.view(len(x), 107648)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Component Classification
for component in components_list:
  net = Net()
  train_dataloader, val_dataloader, test_dataloader = load_data(component)
  trained_model = train_model(net, train_dataloader, val_dataloader)
  evaluate_model(trained_model, test_dataloader, component)

Copying files: 212 files [00:00, 8140.03 files/s]


COM_male
              precision    recall  f1-score   support

           0       1.00      0.88      0.94        74
           1       0.89      1.00      0.94        74

    accuracy                           0.94       148
   macro avg       0.95      0.94      0.94       148
weighted avg       0.95      0.94      0.94       148

---------------------------------------------


Copying files: 398 files [00:00, 8112.46 files/s]


Display_Port
              precision    recall  f1-score   support

           0       1.00      0.90      0.95       139
           1       0.91      1.00      0.95       139

    accuracy                           0.95       278
   macro avg       0.95      0.95      0.95       278
weighted avg       0.95      0.95      0.95       278

---------------------------------------------


Copying files: 610 files [00:00, 8353.69 files/s]


...

USB_2
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       343
           1       0.97      0.97      0.97       343

    accuracy                           0.97       686
   macro avg       0.97      0.97      0.97       686
weighted avg       0.97      0.97      0.97       686

---------------------------------------------


Copying files: 500 files [00:00, 5715.04 files/s]


USB_3
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       175
           1       1.00      0.89      0.94       175

    accuracy                           0.94       350
   macro avg       0.95      0.94      0.94       350
weighted avg       0.95      0.94      0.94       350

---------------------------------------------


Copying files: 110 files [00:00, 4317.67 files/s]


VGA
              precision    recall  f1-score   support

           0       1.00      0.53      0.69        38
           1       0.68      1.00      0.81        38

    accuracy                           0.76        76
   macro avg       0.84      0.76      0.75        76
weighted avg       0.84      0.76      0.75        76

---------------------------------------------


# Pre-trained

In [ ]:
for component in components_list:
  net = torchvision.models.resnet18(pretrained=True)
  # change output layer to 1 for binary classification
  net.fc = nn.Linear(512, 1)
  train_dataloader, val_dataloader, test_dataloader = load_data(component)
  trained_model = train_model(net, train_dataloader, val_dataloader)
  evaluate_model(trained_model, test_dataloader, component)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Copying files: 212 files [00:00, 5636.86 files/s]


COM_male
              precision    recall  f1-score   support

           0       1.00      0.95      0.97        74
           1       0.95      1.00      0.97        74

    accuracy                           0.97       148
   macro avg       0.97      0.97      0.97       148
weighted avg       0.97      0.97      0.97       148

---------------------------------------------


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Copying files: 398 files [00:00, 4735.70 files/s]
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a w

...

USB_2
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       343
           1       0.99      1.00      1.00       343

    accuracy                           1.00       686
   macro avg       1.00      1.00      1.00       686
weighted avg       1.00      1.00      1.00       686

---------------------------------------------


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Copying files: 500 files [00:00, 5036.46 files/s]
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a w

USB_3
              precision    recall  f1-score   support

           0       1.00      0.91      0.95       175
           1       0.92      1.00      0.96       175

    accuracy                           0.95       350
   macro avg       0.96      0.95      0.95       350
weighted avg       0.96      0.95      0.95       350

---------------------------------------------


Copying files: 110 files [00:00, 4000.84 files/s]


VGA
              precision    recall  f1-score   support

           0       1.00      0.79      0.88        38
           1       0.83      1.00      0.90        38

    accuracy                           0.89        76
   macro avg       0.91      0.89      0.89        76
weighted avg       0.91      0.89      0.89        76

---------------------------------------------


In [ ]:
# load ResNet18 model and fine-tuned weights
def load_trained_model(component):
  net = torchvision.models.resnet18(pretrained=True)
  net.fc = nn.Linear(512, 1)
  net.load_state_dict(torch.load(f"/content/drive/MyDrive/DeepVis/PretrainedBinary/{component}.pth", map_location=torch.device('cuda')))
  return net